In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [82]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
from torchvision.ops import sigmoid_focal_loss
from tools.loss import *

In [73]:
preds = torch.rand(4, 2, 200, 200)
logits = torch.logit(preds)
target = torch.randint(0, 2, (4, 2, 200, 200)).float()

In [61]:
F.binary_cross_entropy(preds, target, reduction='mean')

tensor(0.9988)

In [60]:
F.binary_cross_entropy_with_logits(logits, target, reduction='mean')

tensor(0.9988)

In [68]:
sigmoid_focal_loss(logits, target, alpha=-1, gamma=5, reduction='mean')

tensor(0.4082)

In [85]:
FocalLoss()(logits, target)

tensor(0.6175)

In [87]:
new_focal_loss(logits, target, n=2).mean()

tensor(0.6175)

In [92]:
uce_loss(preds+1, target).mean()

tensor(0.6949)

In [90]:
u_focal_loss(preds+1, target).mean()

tensor(0.4898)